# Descarga de los datos formateados

In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown

files_to_download = {
    "formatted_train.json": "1Fm7aPdCv6bguP7UgoCqdUVcm8xpDo18_",
    "formatted_test.json": "1a4YeF--Sks7WA1ZIQL2zDZx4teKk7p4m",
    "formatted_validation.json": "1PC9OhZhNZt8lFO9wifhydy0BrIYe8Dkm"
}

for destination, file_id in files_to_download.items():
    gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Fm7aPdCv6bguP7UgoCqdUVcm8xpDo18_
From (redirected): https://drive.google.com/uc?id=1Fm7aPdCv6bguP7UgoCqdUVcm8xpDo18_&confirm=t&uuid=9ab6e425-7ce3-4570-b3fe-4ec63db02e34
To: /content/formatted_train.json

  0%|          | 0.00/237M [00:00<?, ?B/s]
  2%|▏         | 4.72M/237M [00:00<00:14, 16.6MB/s]
  9%|▊         | 20.4M/237M [00:00<00:03, 63.2MB/s]
 13%|█▎        | 30.4M/237M [00:00<00:04, 49.6MB/s]
 17%|█▋        | 41.4M/237M [00:00<00:03, 63.4MB/s]
 21%|██        | 50.3M/237M [00:00<00:03, 51.9MB/s]
 25%|██▌       | 59.2M/237M [00:01<00:04, 44.0MB/s]
 32%|███▏      | 76.0M/237M [00:01<00:03, 42.9MB/s]
 38%|███▊      | 90.2M/237M [00:01<00:02, 56.9MB/s]
 42%|████▏     | 98.6M/237M [00:01<00:02, 51.3MB/s]
 45%|████▍     | 106M/237M [00:02<00:02, 55.9MB/s] 
 48%|████▊     | 114M/237M [00:02<00:02, 46.8MB/s]
 51%|█████     | 120M/237M [00:02<00:03, 34.6MB/s]
 60%|██████    | 143M/237M [00:02<00:01, 50.0MB/s]
 68%|██████▊   |

# Implementación del modelo

In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from unsloth.chat_templates import get_chat_template, standardize_sharegpt, train_on_responses_only
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
with open("formatted_train.jsonl", "r") as f:
    data = [json.loads(line)["interaction"] for line in f]

dataset = Dataset.from_dict({"conversations": data})

In [ ]:
max_seq_length = 2048

def initialize_model_and_tokenizer():
    base_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_name,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None
    )
    tokenizer = get_chat_template(
        tokenizer,
        chat_template="llama-3.1"
    )

    return model, tokenizer

In [ ]:
def preprocess_dataset(dataset, tokenizer):
    def formatting_prompts_func(examples):
        convos = examples["conversations"]
        texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
        return {"text": texts}

    formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
    return formatted_dataset

In [ ]:
used_seed = 3407

# Entrenamiento

In [ ]:
with open("formatted_train.json", "r") as f:
    data = json.load(f)

dataset_second_train = Dataset.from_dict({"conversations": data})

In [ ]:
training_configuration = {
    "lr_scheduler_type": "linear",
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "max_steps": 200,
    "output_dir": "second_train_output",
    "learning_rate": 2e-4
}

In [ ]:
model_second_train, tokenizer_second_train = initialize_model_and_tokenizer()

==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
formatted_dataset_second_train = preprocess_dataset(dataset_second_train, tokenizer_second_train)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model=model_second_train,
    tokenizer=tokenizer_second_train,
    train_dataset=formatted_dataset_second_train,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        learning_rate=5e-5,
        lr_scheduler_type=training_configuration["lr_scheduler_type"],
        per_device_train_batch_size=training_configuration["per_device_train_batch_size"],
        gradient_accumulation_steps=training_configuration["gradient_accumulation_steps"],
        num_train_epochs=1,
        max_steps=training_configuration["max_steps"],
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        warmup_steps=5,
        output_dir=training_configuration["output_dir"],
        seed=used_seed,
    )
)

Map (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
2.635 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 24,313,856
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mamunoz42 (mamunoz42-pontificia-universidad-cat-lica-de-chile). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.243200
2,2.256100
3,2.164400
4,2.326300
5,2.342700
6,2.176300
7,2.172100
8,2.191600
9,2.080100
10,2.105600


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

373.2538 seconds used for training.
6.22 minutes used for training.
Peak reserved memory = 3.658 GB.
Peak reserved memory for training = 1.023 GB.
Peak reserved memory % of max memory = 9.246 %.
Peak reserved memory for training % of max memory = 2.586 %.


In [ ]:
model_second_train.save_pretrained("lora_model_second_train")
tokenizer_second_train.save_pretrained("lora_model_second_train")

('lora_model_second_train/tokenizer_config.json',
 'lora_model_second_train/special_tokens_map.json',
 'lora_model_second_train/tokenizer.json')

In [ ]:
!zip -r lora_model_second_train.zip lora_model_second_train/

  adding: lora_model_second_train/ (stored 0%)
  adding: lora_model_second_train/tokenizer.json (deflated 85%)
  adding: lora_model_second_train/adapter_config.json (deflated 54%)
  adding: lora_model_second_train/README.md (deflated 66%)
  adding: lora_model_second_train/tokenizer_config.json (deflated 94%)
  adding: lora_model_second_train/adapter_model.safetensors (deflated 8%)
  adding: lora_model_second_train/special_tokens_map.json (deflated 71%)


# Carga del modelo entrenado

In [ ]:
destination = "lora_model_second_train.zip"
gdown.download(f"https://drive.google.com/uc?id=1eHv0yWdVjgq-23AO_HcWR6ImoKyJtiL9", destination, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1eHv0yWdVjgq-23AO_HcWR6ImoKyJtiL9
From (redirected): https://drive.google.com/uc?id=1eHv0yWdVjgq-23AO_HcWR6ImoKyJtiL9&confirm=t&uuid=0df34e24-6f29-4b3f-9a4b-2b7d1128699c
To: /content/lora_model_second_train.zip

  0%|          | 0.00/92.5M [00:00<?, ?B/s]
  5%|▌         | 4.72M/92.5M [00:00<00:08, 9.78MB/s]
 28%|██▊       | 25.7M/92.5M [00:00<00:02, 29.6MB/s]
 37%|███▋      | 34.1M/92.5M [00:01<00:01, 35.2MB/s]
 46%|████▌     | 42.5M/92.5M [00:01<00:01, 32.9MB/s]
 55%|█████▍    | 50.9M/92.5M [00:01<00:01, 36.9MB/s]
 64%|██████▍   | 59.2M/92.5M [00:01<00:01, 32.9MB/s]
 73%|███████▎  | 67.6M/92.5M [00:01<00:00, 40.0MB/s]
100%|██████████| 92.5M/92.5M [00:02<00:00, 38.7MB/s]


'lora_model_second_train.zip'

In [ ]:
!unzip -q lora_model_second_train.zip -d ./

# Testeo del modelo

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model_train",
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

In [ ]:
with open("formatted_test.json", "r") as f:
    data = json.load(f)

dataset = Dataset.from_dict({"conversations": data})

In [ ]:
formatted_test_dataset = preprocess_dataset(dataset, tokenizer)

Map:   0%|          | 0/2277 [00:00<?, ? examples/s]

In [ ]:
%%capture
!pip install tqdm

In [ ]:
percentage = 0.2
sample_size = int(len(formatted_test_dataset) * percentage)
subset_test_dataset = formatted_test_dataset.shuffle(seed=42).select(range(sample_size))

print(f"Usando {sample_size} ejemplos para la evaluación.")

Usando 455 ejemplos para la evaluación.


In [ ]:
from tqdm import tqdm

generation_args = {
    "max_new_tokens": 100,
    "temperature": 0.3,
    "use_cache": True,
    "top_p": 0.9,
    "top_k": 50,
}

predictions = []
references = []

for example in tqdm(subset_test_dataset):
    conversation = example["conversations"]
    system_message = conversation[0]
    user_assistant_turns = conversation[1:-2]

    max_turns = 10
    truncated_turns = user_assistant_turns[-max_turns:]

    messages = [system_message] + truncated_turns
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    print(f"Inputs shape: {inputs.shape}")

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs, **generation_args)

    generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    if "assistant" in generated_response:
        generated_response = generated_response.split("assistant")[-1].strip()

    predictions.append(generated_response)

    references.append(conversation[-2]["content"])



  0%|          | 0/455 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Inputs shape: torch.Size([1, 891])



  0%|          | 1/455 [00:06<52:23,  6.92s/it]

Inputs shape: torch.Size([1, 971])



  0%|          | 2/455 [00:11<39:41,  5.26s/it]

...


100%|█████████▉| 453/455 [28:52<00:07,  3.68s/it]

Inputs shape: torch.Size([1, 899])



100%|█████████▉| 454/455 [28:56<00:03,  3.66s/it]

Inputs shape: torch.Size([1, 831])



100%|██████████| 455/455 [28:59<00:00,  3.82s/it]


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

bleu_score = corpus_bleu(
    [[ref] for ref in references],
    predictions
)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.5438235127832646


In [ ]:
%%capture
!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]

rouge1 = sum([score["rouge1"].fmeasure for score in rouge_scores]) / len(rouge_scores)
rouge2 = sum([score["rouge2"].fmeasure for score in rouge_scores]) / len(rouge_scores)
rougeL = sum([score["rougeL"].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"ROUGE-1: {rouge1}")
print(f"ROUGE-2: {rouge2}")
print(f"ROUGE-L: {rougeL}")

ROUGE-1: 0.4875301201572035
ROUGE-2: 0.23844141275577627
ROUGE-L: 0.3649153520526568


In [ ]:
from collections import Counter

from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction
import numpy as np


def distinct(seqs):
    """ Calculate intra/inter distinct 1/2. """
    """ Recuperado de https://github.com/PaddlePaddle/models/blob/release/1.6/PaddleNLP/Research/Dialogue-PLATO/plato/metrics/metrics.py"""
    batch_size = len(seqs)
    intra_dist1, intra_dist2 = [], []
    unigrams_all, bigrams_all = Counter(), Counter()
    for seq in seqs:
        unigrams = Counter(seq)
        bigrams = Counter(zip(seq, seq[1:]))
        intra_dist1.append((len(unigrams)+1e-12) / (len(seq)+1e-5))
        intra_dist2.append((len(bigrams)+1e-12) / (max(0, len(seq)-1)+1e-5))

        unigrams_all.update(unigrams)
        bigrams_all.update(bigrams)

    inter_dist1 = (len(unigrams_all)+1e-12) / (sum(unigrams_all.values())+1e-5)
    inter_dist2 = (len(bigrams_all)+1e-12) / (sum(bigrams_all.values())+1e-5)
    intra_dist1 = np.average(intra_dist1)
    intra_dist2 = np.average(intra_dist2)
    return intra_dist1, intra_dist2, inter_dist1, inter_dist2

In [ ]:
import nltk
nltk.download('punkt_tab')

tokenized_text = [nltk.word_tokenize(text) for text in references]
resultado = distinct(tokenized_text)
print(resultado)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


(0.8174602211235673, 0.9857301007313511, 0.11541064977609411, 0.3757125912018547)


In [ ]:
%%capture
!pip install evaluate bert_score

In [ ]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=predictions, references=references, lang="en", model_type="distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
from statistics import mean
print(results.keys())
print(f"F1: {mean(results['f1'])}")
print(f"Precision: {mean(results['precision'])}")
print(f"Recall: {mean(results['recall'])}")

dict_keys(['precision', 'recall', 'f1', 'hashcode'])
F1: 0.8604703283571935
Precision: 0.8617244886827993
Recall: 0.8595911986225254


# Testeo Zero Shot

In [ ]:
max_seq_length = 2048
base_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


 24%|██▍       | 4.19M/17.6M [42:54<2:17:03, 1.63kB/s]


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Llam

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

In [ ]:
with open("formatted_test.json", "r") as f:
    data = json.load(f)

dataset = Dataset.from_dict({"conversations": data})

In [ ]:
formatted_test_dataset = preprocess_dataset(dataset, tokenizer)

Map:   0%|          | 0/2277 [00:00<?, ? examples/s]

In [ ]:
percentage = 0.2
sample_size = int(len(formatted_test_dataset) * percentage)
subset_test_dataset = formatted_test_dataset.shuffle(seed=42).select(range(sample_size))

print(f"Usando {sample_size} ejemplos para la evaluación.")

Usando 455 ejemplos para la evaluación.


In [ ]:
from tqdm import tqdm

generation_args = {
    "max_new_tokens": 100,
    "temperature": 0.3,
    "use_cache": True,
    "top_p": 0.9,
    "top_k": 50,
}

predictions = []
references = []

for example in tqdm(subset_test_dataset):
    conversation = example["conversations"]
    system_message = conversation[0]
    user_assistant_turns = conversation[1:-2]

    max_turns = 10
    truncated_turns = user_assistant_turns[-max_turns:]

    messages = [system_message] + truncated_turns
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    print(f"Inputs shape: {inputs.shape}")

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs, **generation_args)

    generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    if "assistant" in generated_response:
        generated_response = generated_response.split("assistant")[-1].strip()

    predictions.append(generated_response)

    references.append(conversation[-2]["content"])


  0%|          | 0/455 [00:00<?, ?it/s]

Inputs shape: torch.Size([1, 891])


  0%|          | 1/455 [00:04<30:16,  4.00s/it]

Inputs shape: torch.Size([1, 971])


  0%|          | 2/455 [00:07<28:27,  3.77s/it]

...

100%|█████████▉| 453/455 [31:54<00:08,  4.32s/it]

Inputs shape: torch.Size([1, 899])


100%|█████████▉| 454/455 [31:58<00:04,  4.30s/it]

Inputs shape: torch.Size([1, 831])


100%|██████████| 455/455 [32:03<00:00,  4.23s/it]


In [ ]:
bleu_score = corpus_bleu(
    [[ref] for ref in references],
    predictions
)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.3720423190369473


In [ ]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]

rouge1 = sum([score["rouge1"].fmeasure for score in rouge_scores]) / len(rouge_scores)
rouge2 = sum([score["rouge2"].fmeasure for score in rouge_scores]) / len(rouge_scores)
rougeL = sum([score["rougeL"].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"ROUGE-1: {rouge1}")
print(f"ROUGE-2: {rouge2}")
print(f"ROUGE-L: {rougeL}")

ROUGE-1: 0.36443422599813075
ROUGE-2: 0.12167336133694615
ROUGE-L: 0.2351907453172963


In [ ]:
def distinct(seqs):
    """ Calculate intra/inter distinct 1/2. """
    """ Recuperado de https://github.com/PaddlePaddle/models/blob/release/1.6/PaddleNLP/Research/Dialogue-PLATO/plato/metrics/metrics.py"""
    batch_size = len(seqs)
    intra_dist1, intra_dist2 = [], []
    unigrams_all, bigrams_all = Counter(), Counter()
    for seq in seqs:
        unigrams = Counter(seq)
        bigrams = Counter(zip(seq, seq[1:]))
        intra_dist1.append((len(unigrams)+1e-12) / (len(seq)+1e-5))
        intra_dist2.append((len(bigrams)+1e-12) / (max(0, len(seq)-1)+1e-5))

        unigrams_all.update(unigrams)
        bigrams_all.update(bigrams)

    inter_dist1 = (len(unigrams_all)+1e-12) / (sum(unigrams_all.values())+1e-5)
    inter_dist2 = (len(bigrams_all)+1e-12) / (sum(bigrams_all.values())+1e-5)
    intra_dist1 = np.average(intra_dist1)
    intra_dist2 = np.average(intra_dist2)
    return intra_dist1, intra_dist2, inter_dist1, inter_dist2

In [ ]:
nltk.download('punkt_tab')

tokenized_text = [nltk.word_tokenize(text) for text in predictions]
resultado = distinct(tokenized_text)
print(resultado)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


(0.7221576114901389, 0.9508190513729, 0.0907063749434587, 0.28086002097908586)


In [ ]:
bertscore = load("bertscore")
results = bertscore.compute(predictions=predictions, references=references, lang="en", model_type="distilbert-base-uncased")

In [ ]:
print(results.keys())
print(f"F1: {mean(results['f1'])}")
print(f"Precision: {mean(results['precision'])}")
print(f"Recall: {mean(results['recall'])}")

dict_keys(['precision', 'recall', 'f1', 'hashcode'])
F1: 0.8185031487391545
Precision: 0.8018398773539197
Recall: 0.836233556663597
